<a href="https://colab.research.google.com/github/hoihui/pkgs/blob/master/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
# !pip install tensorflow==2.0.0-alpha0
import tensorflow as tf
from tensorflow import keras
keras.__version__,tf.__version__,tf.test.is_gpu_available()

('2.2.4-tf', '2.0.0-alpha0', False)

# Tensors

## Numbers

In [80]:
x=tf.Variable(1,dtype=float)    #init scalar
tf.zeros([2,3])
tf.ones([2,3])

<tf.Tensor: id=1241, shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [ ]:
x.assign(2.)   # so that memory location is overwritten in-place

In [65]:
tf.add([1, 2], [3, 4])
tf.square(5)
tf.reduce_sum([1, 2, 3],axis=None)   #  sum of elements across dimensions
tf.reduce_mean([1, 2, 3],axis=None)  # mean of elements across dimensions
# x, x.item()            # convert to python scalar

<tf.Tensor: id=950, shape=(), dtype=int32, numpy=2>

## Linear Alegra

In [11]:
tf.matmul([[1]], [[2, 3]])

<tf.Tensor: id=25, shape=(1, 2), dtype=int32, numpy=array([[2, 3]], dtype=int32)>

In [18]:
features = tf.random.normal((2,5)) #tensor of size (2,5)
# weights = 
# bias = tf.randn((1,1))
print(features)

tf.Tensor(
[[-0.79568696  2.1542447  -0.16709523 -0.12590285 -1.0148194 ]
 [ 0.4772951   1.001898   -0.22688027 -0.36836457 -1.2996123 ]], shape=(2, 5), dtype=float32)


## Functions

In [68]:
1/(1 + tf.exp(tf.reduce_sum(features)))

<tf.Tensor: id=958, shape=(), dtype=float32, numpy=0.5902318>

## Autograd

In [63]:
x = tf.random.normal((2,2))
print(x.numpy()/2,x.numpy()*2)
with tf.GradientTape(persistent=True) as t:  #persistent=True so that t.gradient can be called multiple times
    t.watch(x)
    y = x*x
    z = tf.reduce_mean(y)

[[-0.7093405  -0.6687705 ]
 [-0.19664665  0.26482087]] [[-2.837362  -2.675082 ]
 [-0.7865866  1.0592835]]


To prove $
\frac{\partial z}{\partial x} = \frac{\partial}{\partial x}\left[\frac{1}{n}\sum_i^n x_i^2\right] = \frac{x}{2}
$ and $\frac{\partial y}{\partial x} = 2x$:

In [64]:
print(t.gradient(z,x))  #=dz/dx
print(t.gradient(y,x))  #=dy/dx
del t  # if persistent=True, remember to release the ref

tf.Tensor(
[[-0.7093405  -0.6687705 ]
 [-0.19664665  0.26482087]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[-2.837362  -2.675082 ]
 [-0.7865866  1.0592835]], shape=(2, 2), dtype=float32)


In [79]:
# Higher order gradients using multiple GradientTape's
x = tf.Variable(3.14159)

with tf.GradientTape() as t:
    with tf.GradientTape() as t2:
        y = x * x * x
    dy_dx = t2.gradient(y, x)
d2y_dx2 = t.gradient(dy_dx, x)

print(dy_dx.numpy(), 3*x.numpy()**2) #d(x^3)/dx=3x^2
print(d2y_dx2.numpy(), 6*x.numpy())  #d(3x^2)/dx=6x

29.608767 29.608765416240203
18.84954 18.84954071044922


## To/from Numpy

Is memory shared?

In [47]:
import numpy as np
a = np.random.rand(2,2)
b = tf.add(a,0)
print(b)
print(b.numpy())

tf.Tensor(
[[0.07790517 0.87036976]
 [0.05034188 0.88710919]], shape=(2, 2), dtype=float64)
[[0.07790517 0.87036976]
 [0.05034188 0.88710919]]


In [48]:
b*=2 # inplace multiplication
a # shows that memory is shared

array([[0.07790517, 0.87036976],
       [0.05034188, 0.88710919]])

# Datasets

[Keras datasets](https://keras.io/datasets/) (all uint8):
* mnist: X.shape is (N, 28, 28); y.shape is (N,) from 0 to 9
* cifar10: X.shape is (N, 3, 32, 32); y.shape is (N,) from 0 to 9
* cifar100: X.shape is (N, 3, 32, 32); y.shape is (N,) of string labels
 * label_mode: "fine" or "coarse".
* imdb: 
* reuters

In [82]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train, 10)# convert class vectors to binary class matrices

Using TensorFlow backend.


In [86]:
# Conform with Convolution2D's requirements for different backends
# X_train_ = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
# X_test_ = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
# input_shape = (img_rows, img_cols, 1)

## Custom Dataset

https://www.tensorflow.org/guide/datasets

In [ ]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write("""Line 1\nLine 2\nLine 3\n  """)

ds_file = tf.data.TextLineDataset(filename)

In [ ]:
# transformation map/batch/shuffle

ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)  #loader

In [ ]:
#iteration

print('Elements of ds_tensors:')
for x in ds_tensors:
    print(x)

print('\nElements in ds_file:')
for x in ds_file:
    print(x)

# Constructing Network

## Prebuilt <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers">Layers</a>

In [24]:
layer = tf.keras.layers.Dense(10)
layer(tf.zeros([10, 5]))  #forward an input tensor
layer.variables # ==.weights; outputs both "weights" and biases
layer.kernel, layer.bias

(<tf.Variable 'dense_5/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.23853138, -0.47463644, -0.40410075,  0.3337345 ,  0.05842251,
         -0.2110303 , -0.37228498, -0.52343273, -0.1935311 ,  0.31415182],
        [-0.47757697,  0.1872893 ,  0.36996263,  0.53191036, -0.41023302,
         -0.36609358, -0.62482274, -0.32694978, -0.46359128, -0.11053449],
        [ 0.5979156 , -0.6117383 ,  0.52128893, -0.20676479, -0.3525084 ,
          0.2735374 , -0.5580343 , -0.06586772,  0.1952563 ,  0.3177566 ],
        [-0.04371983,  0.25442594,  0.5144964 ,  0.03669256, -0.4263901 ,
         -0.47154313,  0.18181372, -0.14908463, -0.6010451 ,  0.37639374],
        [-0.60666305,  0.17177355, -0.03292   , -0.6297653 , -0.30985603,
          0.22744763,  0.08481169,  0.03695041, -0.4091086 ,  0.21218657]],
       dtype=float32)>,
 <tf.Variable 'dense_5/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

## Custom Layer

In [26]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):     # does not know the input shape yet
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs

    def build(self, input_shape):        # called 1st when forwarding; knows the input shape already
        self.kernel = self.add_variable("kernel",
                                        shape=[int(input_shape[-1]),
                                               self.num_outputs])

    def call(self, inp):                 # called 2nd when forwarding
        return tf.matmul(inp, self.kernel)

layer = MyDenseLayer(10)
layer(tf.zeros([10, 5]))  # forwarding: first call .build(), then call .call()
layer.trainable_variables

[<tf.Variable 'my_dense_layer_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.00734526,  0.10645199, -0.5941866 , -0.25046846,  0.22676909,
         -0.55612785, -0.2539965 ,  0.07490993, -0.00648773,  0.41902   ],
        [ 0.40334123,  0.23245806, -0.3132097 ,  0.1817038 ,  0.17335898,
          0.05050379,  0.1570611 , -0.08801335,  0.09774137,  0.41143197],
        [ 0.33316398, -0.00309026,  0.20375234, -0.53965926, -0.50916684,
         -0.2689581 , -0.49531913, -0.16076416,  0.10149634, -0.2719392 ],
        [ 0.08511382, -0.0683853 , -0.2273733 , -0.20938778, -0.2959872 ,
          0.0690949 , -0.40579018, -0.44823477, -0.40371758, -0.5016375 ],
        [ 0.606834  ,  0.3403492 ,  0.08920056, -0.08068699,  0.60987383,
          0.07875824, -0.15887764,  0.28663874,  0.5244569 ,  0.14796907]],
       dtype=float32)>]

In [27]:
# composing multiple layers to a usable layer
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters

        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()

        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()

        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2c(x)
        x = self.bn2c(x, training=training)

        x += input_tensor
        return tf.nn.relu(x)

block = ResnetIdentityBlock(1, [1, 2, 3])
print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.trainable_variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization_v2/gamma:0', 'resnet_identity_block/batch_normalization_v2/beta:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_v2_1/gamma:0', 'resnet_identity_block/batch_normalization_v2_1/beta:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_v2_2/gamma:0', 'resnet_identity_block/batch_normalization_v2_2/beta:0']


## Sequential Building

In [ ]:
from keras.models import Sequential
model = Sequential()

In [ ]:
from keras.layers import Dense
model.add(Dense(512, input_shape=(96,96)))

<a href="https://keras.io/layers/core/#dense">Dense</a>(output_dim, input_shape, init, W_regularizer)
* output_dim: Linear channel's length
* input_shape: only necessary on first layer; otherwise is inferred internally
* init: "glorot_normal"
* W_regularizer=l2(0.1) [from keras.regularizers import l2]
* activation: if specified, equiv to addling the Activation layer as the next item

<a href="https://keras.io/layers/core/#activation">Activation</a>(<a href="https://keras.io/activations/">activation</a>)
* 'relu', 'softplus', 'softsign', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear', 'softmax'(3dim or 2dim)
* custom function with 1 input & 1 output

<a href="https://keras.io/layers/convolutional/#convolution2d">Convolution2D</a>(nb_filter, nb_row, nb_col, init='glorot_uniform', activation='linear', weights=None, border_mode='valid', subsample=(1, 1), dim_ordering='default', W_regularizer=None, b_regularizer=None, activity_regularizer=None, W_constraint=None, b_constraint=None, bias=True)
* Input shape: 4D tensor with shape: (samples, channels, rows, cols) if dim_ordering='th' or 4D tensor with shape: (samples, rows, cols, channels) if dim_ordering='tf'.

MaxPooling2D(pool_size=pool_size)

merge([tower_1, tower_2, tower_3], mode='concat', concat_axis=1)

Flatten()

<a href="https://keras.io/layers/wrappers/#timedistributed">TimeDistributed</a>(Dense(10,input_shape))
* input must be at least 3D (batchSize x timeSteps x DenseInput)

## Functional API ~nngraph

https://keras.io/getting-started/functional-api-guide/

In [ ]:
from keras.layers import Input, TimeDistributed
from keras.layers import Dense, ...

In [ ]:
inputs = Input(shape=(100,), dtype='int32')#Sequential
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(input=inputs, output=predictions)

In [ ]:
x = Input(shape=(dim1, dim2, dim3))
a = TimeDistributed(LSTM(output_dim=10))(x) #apply "LSTM" model to each of dim1's elements
b = LSTM(col_hidden)(a)
prediction = Dense(nb_classes, activation='softmax')(b)
model = Model(input=x, output=prediction)

In [ ]:
model = Model(input=[main_input, auxiliary_input], output=[main_output, auxiliary_output]) #multi-input/outputs
model.compile(optimizer='rmsprop', loss='binary_crossentropy', loss_weights=[1., 0.2])
model.fit([headline_data, additional_data], [labels, labels],nb_epoch=50, batch_size=32)
# if names are provided for the inputs/outputs:
model.compile(optimizer='rmsprop',loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
                          loss_weights={'main_output': 1., 'aux_output': 0.2})
model.fit({'main_input': headline_data, 'aux_input': additional_data},
          {'main_output': labels, 'aux_output': labels},
          nb_epoch=50, batch_size=32)

## Common Model Properties

In [ ]:
model.summary()  #human-readable
model.output_shape # shape of output

In [ ]:
from keras.models import model_from_json
model.save_weights("model.h5", overwrite=True)
with open("model.json", "w") as outfile:  #uncompiled
    json.dump(model.to_json(), outfile)
with open("model.json", "r") as jfile:
    model = model_from_json(json.load(jfile))
model.load_weights("model.h5")

In [ ]:
model.add(Dense(64, activation='relu', name="dense_one"))
model.get_layer("dense_one")  #get layer by name

## Common Layer Properties

In [ ]:
layer = model.layers[0]  #extract the first layer from the list of model's layers
layer.get_weights()[0] # For "Dense": 0 for weights, 1 for bias
layer.set_weights(weights) # sets the weights of the layer from a list of Numpy arrays 
layer.get_config()# returns a dictionary containing the configuration of the layer. 

If a layer has a single node (i.e. if it isn't a shared layer), you can get its input tensor, output tensor, input shape and output shape via:

    layer.input
    layer.output
    layer.input_shape
    layer.output_shape

If the layer has multiple nodes (see: the concept of layer node and shared layers), you can use the following methods:

    layer.get_input_at(node_index)
    layer.get_output_at(node_index)
    layer.get_input_shape_at(node_index)
    layer.get_output_shape_at(node_index)

# Training Network

## Defining loss function

* Binay
  * return a single number as output with `BCEWithLogitsLoss`, or
  * return a probability as output with `BCELoss`
* Multiclass
  * return multiple numbers (eg. from `nn.Linear` output) with `CrossEntropyLoss`, or
  * return multiple probabilities (eg. from `nn.LogSoftmax` output) with `NLLLoss`

In [166]:
model = nn.Sequential(nn.Linear(784, 256),  #could also use class or OrderedDict
                      nn.ReLU(),
                      nn.Linear(256, 10),
                      nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()

images = images.view(64, -1)              #originally 64x1x28x28
loss = criterion( model(images), labels)  #averaged over the batch
print(loss)

tensor(2.3173, grad_fn=<NllLossBackward>)


## backprop

In [167]:
print('first layer grad Before backward pass: \n', model[0].weight.grad)
loss.backward()
print('first layer grad After backward pass: \n', model[0].weight.grad)
nn.utils.clip_grad_norm_(model.parameters(), 5.0)   # gradient clipping

first layer grad Before backward pass: 
 None
first layer grad After backward pass: 
 tensor([[-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003],
        [-0.0058, -0.0058, -0.0058,  ..., -0.0058, -0.0058, -0.0058],
        [-0.0059, -0.0059, -0.0059,  ..., -0.0059, -0.0059, -0.0059],
        ...,
        [-0.0012, -0.0012, -0.0012,  ..., -0.0012, -0.0012, -0.0012],
        [-0.0015, -0.0015, -0.0015,  ..., -0.0015, -0.0015, -0.0015],
        [ 0.0038,  0.0038,  0.0038,  ...,  0.0038,  0.0038,  0.0038]])


## Optimizer

http://pytorch.org/docs/master/optim.html
* `optim.SGD`
* `optim.Adam`

In [171]:
from torch import optim
optimizer = optim.SGD(model.parameters(), lr=0.005)

In [173]:
# One step
optimizer.zero_grad()                   # gradients are accumulated -- clear first
loss = criterion(model(images), labels) # forward a batch
loss.backward()
# print(model[0].weight.grad)           # get the grad of parameters
print('Initial weights - ', model[0].weight)
optimizer.step()                        # update params with accumulated grad
print('Gradient -', model[0].weight.grad)

Initial weights -  Parameter containing:
tensor([[ 0.0225,  0.0054, -0.0342,  ..., -0.0061,  0.0087,  0.0269],
        [-0.0331,  0.0253,  0.0078,  ...,  0.0262,  0.0045, -0.0281],
        [-0.0069,  0.0357, -0.0105,  ...,  0.0260,  0.0166,  0.0045],
        ...,
        [ 0.0043, -0.0352,  0.0160,  ...,  0.0255, -0.0023, -0.0298],
        [ 0.0159, -0.0356, -0.0056,  ...,  0.0277,  0.0312,  0.0060],
        [-0.0139,  0.0074,  0.0274,  ..., -0.0272, -0.0031, -0.0049]],
       requires_grad=True)
Gradient - tensor([[-7.0587e-05, -7.0587e-05, -7.0587e-05,  ..., -7.0587e-05,
         -7.0587e-05, -7.0587e-05],
        [-4.9133e-03, -4.9133e-03, -4.9133e-03,  ..., -4.9133e-03,
         -4.9133e-03, -4.9133e-03],
        [-6.5717e-03, -6.5717e-03, -6.5717e-03,  ..., -6.5717e-03,
         -6.5717e-03, -6.5717e-03],
        ...,
        [-5.8080e-04, -5.8080e-04, -5.8080e-04,  ..., -5.8080e-04,
         -5.8080e-04, -5.8080e-04],
        [-1.1184e-03, -1.1184e-03, -1.1184e-03,  ..., -1.1184e

In [174]:
# Full epochs
for epoch in range(5):
    running_loss = 0
    for images, labels in trainloader:        
        images = images.view(images.shape[0], -1)
        
        optimizer.zero_grad()                     # empty grad
        loss = criterion(model(images), labels)   # forward
        loss.backward()                           # backward        
        optimizer.step()                          # update params with accumulated grad
        
        running_loss += loss.item()               # loss value itself??
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")

Training loss: 0.9859724194129139
Training loss: 0.44501842942827546
Training loss: 0.3749764274432461
Training loss: 0.34324646042163437
Training loss: 0.3232783184154456


In [ ]:
# With early stopping
import tempfile                    # tempfile for saving model
_,weightsfile=tempfile.mkstemp()
#####.....
    if valid_loss <= valid_loss_min:
        torch.save(model.state_dict(), weightsfile)
    else:
        epochs_without_improvement+=1
    if epochs_without_improvement>patience:
        break
model.load_state_dict(torch.load(weightsfile))

## On GPU

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.cuda() #move to cuda
# model.cpu()  #move back to cpu
model.to(device)
images=images.to(device)  #is not inplace

## Keras

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.RMSprop(),metrics=['accuracy']) 

[Compilation parameters](https://keras.io/models/model/):
* [loss](https://keras.io/losses/):
  * `categorical_crossentropy`: model output before sigmoid. y of shape (N,nclass), i.e. one-hot encoded
  * `sparse_categorical_crossentropy`: model output before sigmoid. y of shape (N,), i.e. non-one-hot encoded
  * `binary_crossentropy`: model output before sigmoid. y is a single number
  * 'mse' (mean_squared_error)
  * 'mae'
  * 'mape', squared_hinge, hinge, , kld (kullback_leibler_divergence),
  * 'poisson' [Mean of (predictions - targets * log(predictions))]
  * 'cosine_proximity'
* [optimizer](https://keras.io/optimizers/): common parameters are clipnorm (max allowed gradient 2-norm) and clipvalue (max allowed gradient 1-norm)
  * 'sgd' or SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
  * 'adagrad' or Adagrad(lr=0.01, epsilon=1e-08, decay=0.0)
  * 'rmsprop' or RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
  * 'adadelta' or Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
  * 'adam' or Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
  * 'adamax' or Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
  * 'nadam' or Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
* [metrics](https://keras.io/metrics/): list of metrics to be evaluated by the model during training and testing
  * 'accuracy'
* sample_weight_mode: If you need to do timestep-wise sample weighting (2D weights), set this to "temporal".  None defaults to sample-wise weights (1D). If the model has multiple outputs, you can use a different  sample_weight_mode on each output by passing a dictionary or a list of modes.

In [ ]:
import tempfile
_,weightsfile=tempfile.mkstemp()
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
es=EarlyStopping(monitor='val_loss', patience=patience, verbose=1)
mc=ModelCheckpoint(weightsfile,monitor='val_loss',save_best_only=True,save_weights_only=True,verbose=0)

history = model.fit(x,y,batch_size=32, epochs=10,
                    callbacks=[es,mc],
                    verbose=1,validation_split=0.0,
                    validation_data=None, shuffle=True, class_weight=None, sample_weight=None)
model.load_weights(weightsfile)

[Parameters](https://keras.io/models/model/):

* `x`: Numpy array of training data, or list of Numpy arrays if the model has multiple inputs. If all inputs in the model are named, you can also pass a dictionary mapping input names to Numpy arrays.
* `y`: Numpy array of target data, or list of Numpy arrays if the model has multiple outputs. If all outputs in the model are named, you can also pass a dictionary mapping output names to Numpy arrays.
* `batch_size`: integer. Number of samples per gradient update.
* `epochs`: integer, the number of times to iterate over the training data arrays.
* `verbose`: 0, 1, or 2. Verbosity mode. 0 = silent, 1 = verbose, 2 = one log line per epoch.
* `validation_split`: float between 0 and 1: fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch.
* `validation_data`: data on which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data. This could be a tuple (x_val, y_val) or a tuple (val_x, val_y, val_sample_weights).
* `shuffle`: boolean, whether to shuffle the training data before each epoch.
* `class_weight`: optional dictionary mapping class indices (integers) to a weight (float) to apply to the model's loss for the samples from this class during training. This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
* `sample_weight`: optional array of the same length as x, containing weights to apply to the model's loss for each sample. In the case of temporal data, you can pass a 2D array with shape (samples, sequence_length), to apply a different weight to every timestep of every sample. In this case you should make sure to specify sample_weight_mode="temporal" in compile().
* `callbacks`: list of callbacks to be called during training. See callbacks.

In [ ]:
# General model() class
# https://keras.io/models/model/
evaluate(self, x, y, batch_size=32, verbose=1, sample_weight=None)
predict(self, x, batch_size=32, verbose=0)
train_on_batch(self, x, y, sample_weight=None, class_weight=None)
test_on_batch(self, x, y, sample_weight=None)
predict_on_batch(self, x)
fit_generator(self, generator, samples_per_epoch, nb_epoch, verbose=1, callbacks=[], validation_data=None, nb_val_samples=None, class_weight={}, max_q_size=10, nb_worker=1, pickle_safe=False)
evaluate_generator(self, generator, val_samples, max_q_size=10, nb_worker=1, pickle_safe=False)
predict_generator(self, generator, val_samples, max_q_size=10, nb_worker=1, pickle_safe=False)
get_layer(self, name=None, index=None)

In [ ]:
# Sequential() only ?
# https://keras.io/models/sequential/
predict_classes(self, x, batch_size=32, verbose=1)
predict_proba(self, x, batch_size=32, verbose=1)

# Custom Evaluation

In [ ]:
y_hat = model.predict_classes(x) #Sequential() only
pandas.crosstab(y_hat, y_test)

# Inference

* `with torch.no_grad()`:  no grad tracking
* `model.eval()`        :  zero dropout probability
* `model.train()`       :  back to nonzero original dropout

In [188]:
# inference
with torch.no_grad():             # tells torch does not need to keep track of grad
    model.eval()                  # .eval() mode sets dropout probability to zero
    prob=torch.exp(model(images)) # The last layer is LogSoftmax, not Softmax
model.train()                     # opposite of .eval()
    
print(prob[0])
top_p, top_class = prob[:3].topk(2,dim=1)     # top-k most likely probability and the predicted classes


tensor([1.5433e-01, 1.0443e-03, 3.7916e-01, 5.2210e-02, 3.2306e-04, 1.2050e-02,
        3.9326e-01, 2.8806e-04, 7.2790e-03, 5.5042e-05])
torch.return_types.topk(
values=tensor([[0.3933, 0.3792],
        [0.9888, 0.0055],
        [0.9653, 0.0344]]),
indices=tensor([[6, 2],
        [6, 2],
        [7, 9]]))


In [189]:
# Evaluate on test
running_loss = 0
running_acc = 0
for images, labels in testloader:
    images = images.view(images.shape[0], -1)
    with torch.no_grad():             # tells torch does not need to keep track of grad
        out = model(images)           # forward
        
    loss = criterion(out, labels)     
    prob = torch.exp(out)
    top_p, top_class = prob.topk(1, dim=1)
    equals = (top_class == labels.view(*top_class.shape))
    
    running_loss += loss.item()
    running_acc += torch.mean(equals.type(torch.FloatTensor))   #cast torch.ByteTensor to torch.FloatTensor for taking .mean()

print(f"Test LogLoss:  {running_loss/len(trainloader)}")
print(f"Test Accuracy: {running_acc/len(trainloader)}")

Test LogLoss:  0.31288541873285514
Test Accuracy: 0.9099313616752625


# Save / Load

In [ ]:
model.state_dict() # to ordered dict
torch.save(model.state_dict(), 'filename')      # save
model.load_state_dict( torch.load('filename') ) # load; "model" variable must have the same architecture

Need to manually build a dictionary with all the information you need to compeletely rebuild the model.

## Pretrained Models

https://pytorch.org/docs/stable/torchvision/models.html

most model has `classifier` and `features` part.

For transfer learning, turn of grad for `features` part and attach custom classifier

In [14]:
from collections import OrderedDict
from torch import nn
from torchvision import models
model = models.densenet121(pretrained=True)

In [17]:
for param in model.parameters():      # Freeze parameters of the feature detector
    param.requires_grad = False
    
print(f"old: {model.classifier}")     # seeing the input is 1024, we need to match it in our own classifier
from collections import OrderedDict   # create our own (binary) classifier
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 256)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(256, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))    
model.classifier = classifier
print(f"new: {model.classifier}")

old: Linear(in_features=1024, out_features=1000, bias=True)
new: Sequential(
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=256, out_features=2, bias=True)
  (output): LogSoftmax()
)


# sklearn interface

In [ ]:
import tensorflow.estimator as skflow
from sklearn import datasets, metrics

iris = datasets.load_iris()
classifier = skflow.LinearClassifier(n_classes=3)
classifier.fit(iris.data, iris.target)
score = metrics.accuracy_score(iris.target, classifier.predict(iris.data))
print("Accuracy: %f" % score)

In [ ]:
import tensorflow.contrib.learn as skflow
from sklearn import datasets, metrics, preprocessing

boston = datasets.load_boston()
X = preprocessing.StandardScaler().fit_transform(boston.data)
regressor = skflow.LinearRegressor()
regressor.fit(X, boston.target)
score = metrics.mean_squared_error(regressor.predict(X), boston.target)
print ("MSE: %f" % score)

In [ ]:
import tensorflow.contrib.learn as skflow
from sklearn import datasets, metrics

iris = datasets.load_iris()
classifier = skflow.DNNClassifier(hidden_units=[10, 20, 10], n_classes=3)
classifier.fit(iris.data, iris.target)
score = metrics.accuracy_score(iris.target, classifier.predict(iris.data))
print("Accuracy: %f" % score)

In [ ]:
import tensorflow.contrib.learn as skflow
from sklearn import datasets, metrics

iris = datasets.load_iris()

def my_model(X, y):
    """This is DNN with 10, 20, 10 hidden layers, and dropout of 0.5 probability."""
    layers = skflow.ops.dnn(X, [10, 20, 10])
    return skflow.models.logistic_regression(layers, y)

classifier = skflow.TensorFlowEstimator(model_fn=my_model, n_classes=3)
classifier.fit(iris.data, iris.target)
score = metrics.accuracy_score(iris.target, classifier.predict(iris.data))
print("Accuracy: %f" % score)